In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
!pip install nltk

import itertools
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

# Fonction permettant de rassembler les n mots les plus fréquents d'un fichier texte dans un seul dataframe

def mostcommonwords(path,n):
    file_location = path
    rdd = sc.textFile(file_location)
    # On récupère le contenu des fichiers texte
    line = rdd.take
    lines = rdd.collect()
    words = []
    # On sépare le texte en lignes
    for stuff in lines:
        words.append(stuff.split())
    
    words = list(itertools.chain.from_iterable(words))
    # On sépare les lignes en mots pour ensuite créer le dataframe qui contiendra les mots et leur nombre d'occurences

    dict = {'word':[],
            'occurences':[]}
    df_occurences = pd.DataFrame(dict)

    # Tout d'abord, on assigne à tous les mots une occurence de 1
    for word in words:
        df_occurences.loc[len(df_occurences.index)] = [word, 1]

    # Par la suite, on retire tous les symboles de ponctuation dans le dataframe 
    for thing in [*string.punctuation]:
        df_occurences.drop(df_occurences[df_occurences['word']==thing].index, inplace = True)

    # Enfin, on supprime les mots de liaison tels que 'the', 'an' ou encore 'and'. 
    # Pour l'instant, nous nous sommes limités aux textes anglais
    for bip in stopwords.words("english"):
        df_occurences.drop(df_occurences[df_occurences['word']==bip].index, inplace = True)

    # Pour obtenir le nombre d'occurence de chaque mot unique, on regroupe les mots identiqus 
    # en sommant les occurences puis en les classant par ordre décroissant  
    df_occurences = df_occurences.groupby(['word'],as_index=False).sum().sort_values(by=['occurences'], ascending=False)

    # On ne récupère que les n les plus fréquents
    df_occurences_top = df_occurences[:n]

    # Enfin on réétablite l'index dont l'ordre a été perturbé par le shuffle and sort
    df_occurences_top = df_occurences_top.set_index(pd.Index(range(n)))
    
    return df_occurences_top

df = mostcommonwords('/FileStore/tables/cv998_15691.txt',10)

df.head(10)



In [ ]:
# On applique la fonction à 6 fichiers text contenant des critiques de film en gardant le même nom que celui du texte 

cv994_12270 = mostcommonwords('FileStore/pos/cv994_12270.txt',10)
cv995_21821 = mostcommonwords('FileStore/pos/cv995_21821.txt',10)
cv996_11592 = mostcommonwords('FileStore/pos/cv996_11592.txt',10)
cv997_5046 = mostcommonwords('FileStore/pos/cv997_5046.txt',10)
cv998_14111 = mostcommonwords('FileStore/pos/cv998_14111.txt',10)
cv999_13106 = mostcommonwords('FileStore/pos/cv999_13106.txt',10)

cv997_5046.head(10)

In [ ]:


# Une fois les dataframes créés on peut les rassembler selon les similitudes  

dataframes=[cv994_12270,cv995_21821,cv996_11592,cv997_5046,cv998_14111,cv999_13106]
names=['cv994_12270','cv995_21821','cv996_11592','cv997_5046','cv998_14111','cv999_13106']
# D'abord on assigne aux dataframes un nom correspondant à leur fichier d'origine
def nomenclatura(frames, names):
    i=0
    for df in frames:
        df.attrs['name']=names[i]
        i+=1
    return frames
# On rassemble les noms des dataframes en comités. Les membres de chaque comités sont stockés dans un dictionnaire
def gathering(frames):
    i=1
    d={}
    for x in range(1,7):
        d["Committee {0}".format(x)] = []
    for dfe in frames:
        d['Committee '+str(i)].append(dfe.attrs['name'])
        for dfco in frames:
            comp = pd.merge(dfe,dfco,how='inner',on='word')
            # Si parmi les 10 mots les plus fréquents 3 sont partagés alors on considère que les textes apartiennent au même comité
            if comp.shape[0]<dfco.shape[0] and comp.shape[0]>3:
                d['Committee '+str(i)].append(dfco.attrs['name'])        
        i+=1
    return d

dataframes = nomenclatura(dataframes,names)
Committees = gathering(dataframes)

print(Committees)


In [ ]:
# Une fois les comités établis, on crée des dossiers pour chaque comité où seront stockés les fichiers membres
# Dans chaque dossier on copie les fichiers qui lui sont assignés

def directorycreation(dictionnary):  
    for i in range(1,len(dictionnary)+1):
        dbutils.fs.mkdirs("/FileStore/Committee"+str(i))
        for frame in dictionnary['Committee '+str(i)]:            
            dbutils.fs.cp("/FileStore/pos/"+frame+".txt","/FileStore/Committee"+str(i)+"/"+frame+".txt")   
        i+=1
    return 0

directorycreation(Committees)             

In [ ]:
%fs
ls /FileStore/Committee4